In [4]:
from quarto import objects, extensions
from quarto.players import MinMaxPlayer, RandomPlayer, ProPlayer
import logging
import math
from copy import deepcopy
from tqdm.notebook import tqdm
import numpy as np

In [5]:
def evaluate_minmax(matches = 1000, minmax_level=2, minmax_bound=math.factorial(6) ** 2, pro = False):
    scores = {
        'me': 0,
        'opponent': 0,
        'draw': 0
    }

    home = {
        0: 'me',
        1: 'opponent',
        -1: 'draw'
    }

    away = {
        0: 'opponent',
        1: 'me',
        -1: 'draw'
    }

    player_stats = np.array([0.0 for _ in range(6)])
    max_select = 0.0
    max_place = 0.0

    for _ in tqdm(range(matches // 2), desc='Home games'):
        game = objects.Quarto()
        opponent = RandomPlayer(game)
        me = ProPlayer(game, bound=minmax_level, bound_value=minmax_bound)  if pro else MinMaxPlayer(game, bound=minmax_level, bound_value=minmax_bound) 
        game.set_players((me, opponent))
        winner = home[game.run()]
        scores[winner] += 1
        player_stats += game.get_stats()[0]
        max_select = max(max_select, game.get_stats()[0][2])
        max_place = max(max_place, game.get_stats()[0][5])

    player_stats[2] = max_select
    player_stats[5] = max_place
    logging.warning(f"Stats: {scores}") 
    print(f'Player 0: Avg select {player_stats[0]/player_stats[1]} s (max {player_stats[2]} s), Avg place {player_stats[3]/player_stats[4]} s (max {player_stats[5]} s)')

    for _ in tqdm(range(matches - matches // 2), desc='Away games'):
        game = objects.Quarto()
        opponent = RandomPlayer(game)
        me = ProPlayer(game, bound=minmax_level, bound_value=minmax_bound)  if pro else MinMaxPlayer(game, bound=minmax_level, bound_value=minmax_bound) 
        game.set_players((opponent, me))
        winner = away[game.run()]  
        scores[winner] += 1
        player_stats += game.get_stats()[1]
        max_select = max(max_select, game.get_stats()[0][2])
        max_place = max(max_place, game.get_stats()[0][5])

    player_stats[2] = max_select
    player_stats[5] = max_place
    logging.warning(f"Stats: {scores}")
    print(f'Player 0: Avg select {player_stats[0]/player_stats[1]} s (max {player_stats[2]} s), Avg place {player_stats[3]/player_stats[4]} s (max {player_stats[5]} s)')

    return scores

In [8]:
evaluate_minmax(1000, 3, math.factorial(5)**2, True)

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 1.044850752921013e-05 s (max 0.0011143684387207031 s), Avg place 0.5379901575451423 s (max 3.9988949298858643 s)


Away games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 0.011967013090514391 s (max 0.0012316703796386719 s), Avg place 0.4128381302815297 s (max 3.9988949298858643 s)


{'me': 998, 'opponent': 1, 'draw': 1}

In [4]:
evaluate_minmax(1000, 2, math.factorial(5)**2)
evaluate_minmax(1000, 2, math.factorial(6)**2)
evaluate_minmax(1000, 3, math.factorial(5)**2)

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 0.02199037982687211 s (max 0.15229058265686035 s), Avg place 0.08217641225204757 s (max 1.5503621101379395 s)


Away games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 0.01132107150170111 s (max 0.15229058265686035 s), Avg place 0.06919577470354524 s (max 1.5503621101379395 s)


Home games:   0%|          | 0/500 [00:00<?, ?it/s]

...

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 0.02570855990923025 s (max 0.18204212188720703 s), Avg place 0.5360447245816752 s (max 3.6956911087036133 s)


Away games:   0%|          | 0/500 [00:00<?, ?it/s]

Player 0: Avg select 0.013118882068949726 s (max 0.18204212188720703 s), Avg place 0.4182762289687905 s (max 3.6956911087036133 s)


{'me': 996, 'opponent': 3, 'draw': 1}

In [5]:
game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 3, math.factorial(5)**2), MinMaxPlayer(game, 2, math.factorial(6)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 2, math.factorial(6)**2), MinMaxPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

Winner is 0
Played 8 turns
Player 0: Avg select 0.02750682830810547 s (max 0.11002731323242188 s), Avg place 0.8444941639900208 s (max 2.488685131072998 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.06976926326751709 s (max 0.10202717781066895 s)
Winner is -1
Played 16 turns
Player 0: Avg select 0.01350286602973938 s (max 0.10802292823791504 s), Avg place 0.20910561084747314 s (max 1.3992741107940674 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.5132704377174377 s (max 1.6715977191925049 s)


In [7]:
game = objects.Quarto()
game.set_players((ProPlayer(game, 3, math.factorial(5)**2), MinMaxPlayer(game, 2, math.factorial(6)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 2, math.factorial(6)**2), ProPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

Winner is 0
Played 8 turns
Player 0: Avg select 0.0 s (max 0.0 s), Avg place 0.8806970119476318 s (max 2.5685813426971436 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.07252073287963867 s (max 0.10703182220458984 s)
Winner is 1
Played 11 turns
Player 0: Avg select 0.017670631408691406 s (max 0.10602378845214844 s), Avg place 0.05403909683227539 s (max 0.09416604042053223 s)
Player 1: Avg select 0.0196044921875 s (max 0.0980224609375 s), Avg place 0.3853995402654012 s (max 1.178267240524292 s)
